In [1]:
import glob
import gradio as gr
from Proxy import Proxy
from collections import deque

import html
import os
import re
import time
from pathlib import Path

import markdown
from PIL import Image, ImageOps
from ContextEntry import ContextEntry


def getProxyList():
  return Proxy.GetProxyList()

items = getProxyList()

activeProxy = Proxy("CliveBarker")
#activeProxy.clearContext()
#activeProxy.TabulaRasa()


[18:34:56.948] globalLog: loading model Embeddings.Embeddings
[18:34:56.949] globalLog: model configured
[18:34:56.949] globalLog: loading model Summarizer.Summarizer
[18:34:56.949] globalLog: model configured
[18:34:56.949] globalLog: loading model NER.NER
[18:34:57.746] globalLog: model configured
[18:34:57.746] globalLog: loading model SentimentNuanced.Sentiment
[18:34:57.746] globalLog: model configured
[18:34:57.746] globalLog: loading model SentimentDiscreet.Sentiment
[18:34:57.747] globalLog: model configured
[18:34:58.056] globalLog: loading model ObjectiveDecisory.GGUF
[18:34:58.057] globalLog: model configured
[18:34:58.058] globalLog: loading model ObjectiveDecisory.GGUF
[18:34:58.086] globalLog: b:\PsychoStasis\Proxies\CliveBarker.proxy
[18:34:58.086] globalLog: {'primer': 'a visionary reimagining the boundaries of horror and mythology, driven by the power of imagination to challenge and transcend societal norms, a writer dedicated to the macabre and the uncany. I respond a

HTML Generation

In [2]:
from Logger import globalLogger, LogLevel

cwd = os.getcwd()

#JS loader
workPath = os.path.join(cwd, 'js')
file_list = glob.glob(os.path.join(workPath, 'chat.js'))
for file in file_list:
  with open(file) as f:
      chatScript = f.read()

#CSS loader
workPath = os.path.join(cwd, 'css')
file_list = glob.glob(os.path.join(workPath, 'chatSylte.css'))
filename = file_list[0]
with open(filename) as f:
    chatStyle = f.read()
    


def replace_blockquote(m):
    return m.group().replace('\n', '\n> ').replace('\\begin{blockquote}', '').replace('\\end{blockquote}', '')
    
    
def convert_to_markdown(string):
  
    # Blockquote
    string = re.sub(r'(^|[\n])&gt;', r'\1>', string)
    pattern = re.compile(r'\\begin{blockquote}(.*?)\\end{blockquote}', re.DOTALL)
    string = pattern.sub(replace_blockquote, string)

    # Code
    string = string.replace('\\begin{code}', '```')
    string = string.replace('\\end{code}', '```')
    string = re.sub(r"(.)```", r"\1\n```", string)

    result = ''
    is_code = False
    for line in string.split('\n'):
        if line.lstrip(' ').startswith('```'):
            is_code = not is_code

        result += line
        if is_code or line.startswith('|'):  # Don't add an extra \n for tables or code
            result += '\n'
        else:
            result += '\n\n'

    result = result.strip()
    if is_code:
        result += '\n```'  # Unfinished code block

    # Unfinished list, like "\n1.". A |delete| string is added and then
    # removed to force a <ol> or <ul> to be generated instead of a <p>.
    if re.search(r'(\n\d+\.?|\n\*\s*)$', result):
        delete_str = '|delete|'

        if re.search(r'(\d+\.?)$', result) and not result.endswith('.'):
            result += '.'

        result = re.sub(r'(\n\d+\.?|\n\*\s*)$', r'\g<1> ' + delete_str, result)

        html_output = markdown.markdown(result, extensions=['fenced_code', 'tables'])
        pos = html_output.rfind(delete_str)
        if pos > -1:
            html_output = html_output[:pos] + html_output[pos + len(delete_str):]
    else:
        html_output = markdown.markdown(result, extensions=['fenced_code', 'tables'])

    # Unescape code blocks
    pattern = re.compile(r'<code[^>]*>(.*?)</code>', re.DOTALL)
    html_output = pattern.sub(lambda x: html.unescape(x.group()), html_output)

    return html_output    
    
def generateMessageHTML(message: ContextEntry):
  role = message.role
  content = message.content
  roleName = message.roleName
  
  imagePath= os.path.join(workPath, "assets", f"{roleName}.png")
  imagePath = glob.glob(imagePath)
  if(os.path.isfile(imagePath[0])):
    imagePath = f"<img src='file/{imagePath[0]}' alt='{roleName}'>"
  else:
    imagePath = ""
  
  if(content):
    if(role == "user"):
      template = f'''
                      <div class="message">
                      <div class="circle-you">
                        {imagePath}
                      </div>
                      <div class="text">
                        <div class="username">
                          {roleName}
                        </div>
                        <div class="message-body">
                          {convert_to_markdown(content)}
                        </div>
                      </div>
                    </div>
      '''
    else:
      template = f'''
                    <div class="message">
                    <div class="circle-bot">
                      {imagePath}
                    </div>
                    <div class="text">
                      <div class="username">
                        {roleName}
                      </div>
                      <div class="message-body">
                        {convert_to_markdown(content)}
                      </div>
                    </div>
                  </div>
      '''
    return template
  else:
    return None
  
  

def generateHTML():
  html = f'<div class="chat" id="chat"><div class="messages">'
  
  history = deque(activeProxy.context.messageHistory)
   
  while len(history) > 0:
    message = history.popleft()
    messageHtml = generateMessageHTML(message)
    if(messageHtml):
      html += messageHtml
  
  html += f'''</div></div>'''
  
  
  return html

def respond(message):
  if(message):
    activeProxy.ReceiveMessage(message=message) 
  html = generateHTML()
  
  return "", html, globalLogger.GenerateHTML()

   


In [3]:
scrollButton = """// Create the button element
const button = document.createElement('button');
button.textContent = 'Scroll';
button.classList.add('scrollButton');

// Define the scrollToBottom function
function scrollToBottom() {
  const element = document.getElementById('chat');
  if (element) {
    element.scrollTop = element.scrollHeight;
    console.log('scrolled');
  }
  else {
    console.log('element not found');
  }
}

// Attach the click event listener to the button
button.addEventListener('click', scrollToBottom);

// Append the button to the document body (or any other desired location)
document.body.appendChild(button);"""


tmpLogHtml ="""
<details>
  <summary class="globalLog">Error Log</summary>
  <p class="globalLog logEntry">This is the first paragraph inside the foldable area.</p>
  <p class="errorLog logEntry">This is the second paragraph inside the foldable area.</p>
  <p class="thoughtLog logEntry">This is the third paragraph inside the foldable area.</p>
</details>
"""

In [4]:
 
with gr.Blocks(css=chatStyle) as chat:
  chat.load(lambda: None, None, None, js=f'() => {{{scrollButton}}}')
  gr.Markdown("# PsychoStasis")
  with gr.Row():
    with gr.Column():
      with gr.Row():
        with gr.Column(scale=4, variant='panel'):
          mainChat = gr.HTML(value = generateHTML(), elem_id="chat_container")
        with gr.Column(scale=1):
          placeholder = gr.HTML(value="")          
          
      txtPrompt = gr.Textbox(label="Prompt", elem_id="prompt")
  with gr.Row():
    with gr.Accordion("Console", open=False) as console:
      consoleLogger = gr.HTML(label="Logger", elem_id="logger", value=globalLogger.GenerateHTML())     
  with gr.Row():  
    txtPrompt.submit(respond, inputs=[txtPrompt], outputs=[txtPrompt, mainChat, consoleLogger])
  
chat.launch(allowed_paths=["."])

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[18:35:07.550] authoritativeLog: Message removed: As an author who often challenges gender norms, I'd encourage considering a non-binary character for the role of the artist. This could add depth and complexity to the narrative by exploring the intersections between identity, creativity, and societal expectations. The artist might be particularly drawn to or inspired by the messianic figure as they awaken to their own potential, further amplifying the story's themes.
[18:35:07.551] cognitiveLog: Running processes in context messageReceived
[18:35:07.551] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:35:07.551] cognitiveLog: Context filtered procs: RefreshMemory
[18:35:07.551] cognitiveLog: Proxy and common filtered procs: RefreshMemory
[18:35:07.551] globalLog: RefreshMemory uncommited: len(history) -- frequency: 0
[18:35:07.551] cognitiveLog: Running RefreshMemory with local context 1 and frequency

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


[18:35:13.760] globalLog: model CBG activated
[18:35:13.879] globalLog: model CBG deactivated
[18:35:13.879] cognitiveLog: Running processes in context beforeGenerateAnswer
[18:35:13.879] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:35:13.879] cognitiveLog: Context filtered procs: 
[18:35:13.879] cognitiveLog: Proxy and common filtered procs: 
[18:35:13.879] globalLog: LoRa: B:/PsychoStasis/lora/CliveBarker.gguf
[18:35:13.879] globalLog: {'model_path': 'B:/PsychoStasis/Training/carbonbeagle-11b-truthy.Q8_0.gguf', 'lora_base': 'B:/PsychoStasis/Training/carbonbeagle-11b-truthy.Q8_0.gguf', 'n_ctx': 4096, 'n_threads': 20, 'n_threads_batch': 1, 'n_batch': 512, 'use_mmap': True, 'use_mlock': True, 'mul_mat_q': True, 'numa': True, 'n_gpu_layers': -1, 'rope_freq_base': 10000.0, 'tensor_split': None, 'rope_freq_scale': 1.0, 'chat_format': 'chatml'}


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


[18:35:22.504] globalLog: model CBG activated
[18:35:22.504] globalLog: performing inference with CBG
[18:35:36.097] cognitiveLog: Running processes in context afterGenerateAnswer
[18:35:36.097] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:35:36.097] cognitiveLog: Context filtered procs: 
[18:35:36.097] cognitiveLog: Proxy and common filtered procs: 
[18:35:36.105] globalLog: context commited
[18:35:36.106] cognitiveLog: Running processes in context afterMessageReceived
[18:35:36.106] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:35:36.106] cognitiveLog: Context filtered procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory
[18:35:36.106] cognitiveLog: Proxy and common filtered procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicM

Llama.generate: prefix-match hit


[18:36:26.663] cognitiveLog: Running processes in context afterGenerateAnswer
[18:36:26.663] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:36:26.663] cognitiveLog: Context filtered procs: 
[18:36:26.663] cognitiveLog: Proxy and common filtered procs: 
[18:36:26.671] globalLog: context commited
[18:36:26.672] cognitiveLog: Running processes in context afterMessageReceived
[18:36:26.672] cognitiveLog: All procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory|RefreshMemory
[18:36:26.672] cognitiveLog: Context filtered procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory
[18:36:26.672] cognitiveLog: Proxy and common filtered procs: CommitToAbstractMemory|CommitToEpisodicMemory|CommitToSummaryMemory|CommitToThematicMemory
[18:36:26.672] globalLog: CommitToAbstractMemory uncommited: len(history) -- frequency: 0
[18:36

Your max_length is set to 256, but your input_length is only 97. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


[18:36:30.543] globalLog: model Summarizer.Summarizer activated
[18:36:31.513] globalLog: model Embeddings.Embeddings deactivated
[18:36:31.521] globalLog: context commited
[18:37:09.074] authoritativeLog: removing last message: <<<<
[18:37:09.074] authoritativeLog: Message removed: In my experience, the gender of characters is often less significant than their personal journey and how they interact with others. However, for this artist character, I'd suggest a female to add diversity in perspective as she navigates her creative process and encounters the Messiah's impact on the world. This could also provide interesting dynamics when exploring themes of motherhood and nurturing within the context of an apathetic messianic figure.
[18:37:09.074] authoritativeLog: Message removed: /A
[18:37:09.074] authoritativeLog: last 2 messages removed
[18:37:09.082] globalLog: context commited
[18:37:35.121] authoritativeLog: removing last message: <<
[18:37:35.121] authoritativeLog: Message remove